<a href="https://colab.research.google.com/github/SangMini2/Deep_Learning_From_Scatch/blob/main/11_23_STUDY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

T,H = 5,4
hs = np.random.randn(T,H)
a = np.array([0.8, 0.1, 0.03, 0.05 , 0.02])

ar = a.reshape(5,1).repeat(4, axis = 1)
print(ar.shape)

(5, 4)


In [2]:
print(ar)

[[0.8  0.8  0.8  0.8 ]
 [0.1  0.1  0.1  0.1 ]
 [0.03 0.03 0.03 0.03]
 [0.05 0.05 0.05 0.05]
 [0.02 0.02 0.02 0.02]]


In [3]:
# 이해를 돕기위해 돌려본 코드
a = np.array([0.8, 0.1, 0.03, 0.05 , 0.02])

ab = a.reshape(5,1)
ab

array([[0.8 ],
       [0.1 ],
       [0.03],
       [0.05],
       [0.02]])

In [4]:
hs.shape

(5, 4)

hs와 a를 계산할 수 있게 하기 위해서 (5,4)shape인 ar을 만들어 주어 계산합니다.

In [5]:
t = hs * ar
print(t)
print(t.shape)

[[ 0.29645059  0.77945308 -0.20532613  0.21355725]
 [-0.01501181  0.08486962  0.10491732  0.08409903]
 [-0.01262321  0.00786522  0.06345055  0.00167015]
 [-0.05114915  0.02830944  0.01098248 -0.05419054]
 [-0.00840586  0.02440166  0.01971432 -0.01291718]]
(5, 4)


In [6]:
c = np.sum(t, axis = 0) # axis=0 의 의미는 세로로 다 더해준다는 의미입니다.
print(c)
print(c.shape)

[ 0.20926055  0.92489902 -0.00626146  0.23221871]
(4,)


미니배치 처리용 가중합 구현

사실 3차원부터 어디에 해당하는 것인지 이해가 잘 안됩니다..

In [7]:
N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H)
a = np.random.randn(N, T)
ar = a.reshape(N, T, 1).repeat(H, axis = 2)


In [8]:
ar

array([[[-0.98644345, -0.98644345, -0.98644345, -0.98644345],
        [-0.15947935, -0.15947935, -0.15947935, -0.15947935],
        [-0.82399205, -0.82399205, -0.82399205, -0.82399205],
        [ 2.28430434,  2.28430434,  2.28430434,  2.28430434],
        [-2.09273095, -2.09273095, -2.09273095, -2.09273095]],

       [[ 0.34553061,  0.34553061,  0.34553061,  0.34553061],
        [ 0.1945381 ,  0.1945381 ,  0.1945381 ,  0.1945381 ],
        [-0.19007739, -0.19007739, -0.19007739, -0.19007739],
        [-0.11370091, -0.11370091, -0.11370091, -0.11370091],
        [-2.08842488, -2.08842488, -2.08842488, -2.08842488]],

       [[-0.04332303, -0.04332303, -0.04332303, -0.04332303],
        [-0.16773744, -0.16773744, -0.16773744, -0.16773744],
        [ 0.09233852,  0.09233852,  0.09233852,  0.09233852],
        [-0.89673473, -0.89673473, -0.89673473, -0.89673473],
        [-1.00565411, -1.00565411, -1.00565411, -1.00565411]],

       [[-1.01486857, -1.01486857, -1.01486857, -1.01486857],
  

그림 8-11을 보면, 지금까지의 과정들이 계산과 함께 나와있습니다.

역전파를 구해보겠습니다.

'sum'과 'repeat'은 서로의 역전파입니다.

다시 얘기하자면 'sum'의 역전파는 'repeat'이고, 'repeat'의 역전파는 'sum'입니다.

In [9]:
class WeightSum :
  def __init__(self) :
    self.params, self.grads = [], []
    self.cache = None
  
  def forward(self, hs, a) :
    N, T, H = hs.shape

    ar = a.reshape(N, T, 1).repeat(H, axis=2)
    t = hs * ar
    c = np.sum(t, axis = 1)

    self.cache = (hs, ar)
    return c
  
  def backward(self, dc) :
    hs, ar = self.cache
    N, T, H = hs.shape

    dt = dc.reshape(N, 1, H).repeat(T, axis=1) #그림 8-11에서의 마지막 sum에서의 역전파
    dar = dt * hs
    dhs = dt * ar
    da = np.sum(dar, axis = 2) # repeat의 역전파

    return dhs, da

다시 ppt로 돌아가겠습니다.

In [10]:
import sys

In [11]:
cd /content/drive/MyDrive/deep-learning-from-scratch-2/

/content/drive/MyDrive/deep-learning-from-scratch-2


In [12]:
from common.layers import Softmax


In [13]:
N, T, H = 10, 5, 4
hs = np.random.randn(N, T, H)
h = np.random.randn(N, H)
hr = h.reshape(N,1,H).repeat(T, axis = 1)

t = hs * hr
print(t.shape)

s = np.sum(t, axis = 2)
print(s.shape)

softmax = Softmax()
a = softmax.forward(s) #가중치
print(a.shape)

(10, 5, 4)
(10, 5)
(10, 5)


In [14]:
class AttentionWeight :
  def __init__(self) :
    self.params, self.grads = [], []
    self.softmax = Softmax()
    self.cache = None
  
  def forward(self, hs, a) :
    N, T, H = hs.shape

    hr = a.reshape(N, 1, H).repeat(T, axis = 1)
    t = hs * hr
    s = np.sum(t, axis = 2)
    a = self.softmax.forward(s)

    self.cache = (hs, hr)
    return a
  
  def backward(self, da) :
    hs, hr = self.cache
    N, T, H = hs.shape

    ds = self.softmax.backward(da)
    dt = ds.reshape(N, T, 1).repeat(H, axis = 2)
    dhs = dt * hr
    dhr = dt * hs
    dh = np.sum(dhr, axis = 1)

    return dhs, dh

In [15]:
class Attention :
  def __init__(self) :
    self.params, self.grads = [], []
    self.attention_weight_layer = AttentionWeight()
    self.weight_sum_layer = WeightSum()
    self.attention_weight = None
  
  def forward(self, hs, h) :
    a = self.attention_weight_layer.forward(hs, h)
    out = self.weight_sum_layer.forward(hs, a)
    self.attention_weight = a #가중치를 a로 두는,,
    return out
  
  def backward(self, dout) :
    dhs0, da = self.weight_sum_layer.backward(dout)
    dhs1, dh = self.attention_weight_layer.backward(da)
    dhs = dhs0 + dhs1
    return dhs, dh

Attention 계층을 우리는 LSTM계층과 Affine계층 사이에 삽입하면 됩니다.

(그림 8-18 참고)

그림 8-18을 보면 LSTM 계층의 은닉 상태 벡터를 Affine 계층에 입력합니다. 이는 앞 장에서 본 Decoder의 개선으로부터 자연스럽게 확장된 것으로 볼 수 있습니다.

In [16]:
class TimeAttention:
    def __init__(self):
        self.params, self.grads = [], []
        self.layers = None
        self.attention_weights = None

    def forward(self, hs_enc, hs_dec):
        N, T, H = hs_dec.shape
        out = np.empty_like(hs_dec)
        self.layers = []
        self.attention_weights = []

        for t in range(T):
            layer = Attention()
            out[:, t, :] = layer.forward(hs_enc, hs_dec[:,t,:])
            self.layers.append(layer)
            self.attention_weights.append(layer.attention_weight)

        return out

    def backward(self, dout):
        N, T, H = dout.shape
        dhs_enc = 0
        dhs_dec = np.empty_like(dout)

        for t in range(T):
            layer = self.layers[t]
            dhs, dh = layer.backward(dout[:, t, :])
            dhs_enc += dhs
            dhs_dec[:,t,:] = dh

        return dhs_enc, 

In [17]:
from ch07.seq2seq import Encoder, Seq2seq

class AttentionEncoder(Encoder) :
  def forward(self, xs) :
    xs = self.embed.forward(xs)
    hs = self.lstm.forward(xs)
    return xs
    
  def backward(self, dhs) :
    dout = self.lstm.backward(dhs)
    dout = self.embed.backward(dout)
    return dout



In [19]:
from common.time_layers import TimeEmbedding, TimeLSTM, TimeAffine

# 코드를 이해 못했습니다.. 아직..

class AttentionDecoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        affine_W = (rn(2*H, V) / np.sqrt(2*H)).astype('f')
        affine_b = np.zeros(V).astype('f')

        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
        self.attention = TimeAttention()
        self.affine = TimeAffine(affine_W, affine_b)
        layers = [self.embed, self.lstm, self.attention, self.affine]

        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, xs, enc_hs):
        h = enc_hs[:,-1]
        self.lstm.set_state(h)

        out = self.embed.forward(xs)
        dec_hs = self.lstm.forward(out)
        c = self.attention.forward(enc_hs, dec_hs)
        out = np.concatenate((c, dec_hs), axis=2)
        score = self.affine.forward(out)

        return score

    def backward(self, dscore):
        dout = self.affine.backward(dscore)
        N, T, H2 = dout.shape
        H = H2 // 2

        dc, ddec_hs0 = dout[:,:,:H], dout[:,:,H:]
        denc_hs, ddec_hs1 = self.attention.backward(dc)
        ddec_hs = ddec_hs0 + ddec_hs1
        dout = self.lstm.backward(ddec_hs)
        dh = self.lstm.dh
        denc_hs[:, -1] += dh
        self.embed.backward(dout)

        return denc_hs

    def generate(self, enc_hs, start_id, sample_size):
        sampled = []
        sample_id = start_id
        h = enc_hs[:, -1]
        self.lstm.set_state(h)

        for _ in range(sample_size):
            x = np.array([sample_id]).reshape((1, 1))

            out = self.embed.forward(x)
            dec_hs = self.lstm.forward(out)
            c = self.attention.forward(enc_hs, dec_hs)
            out = np.concatenate((c, dec_hs), axis=2)
            score = self.affine.forward(out)

            sample_id = np.argmax(score.flatten())
            sampled.append(sample_id)

        return sampled